In [1]:
import os
import pandas as pd
from tabula import read_pdf

In [2]:
def column_splitter(column_name, pdf):
    prices = pdf[column_name].str.split(' ', expand = True)
    liter_price = prices[0]
    galon_price = prices[1]
    
    pdf.drop(column_name, axis = 1, inplace=True)
    return liter_price, galon_price

In [3]:
def file_parser(file_path):
    pdf = read_pdf(file_path, pandas_options = {'skiprows': 1})
    pdf.columns = pd.Index(['company', 'regular_gas', 'premium_gas', 'gas_date', 'diesel', 'diesel_date'])
    
    pdf.fillna(method='ffill', inplace=True)

    pdf['liter_regular'], pdf['galon_regular'] = column_splitter('regular_gas', pdf)
    pdf['liter_premium'], pdf['galon_premium'] = column_splitter('premium_gas', pdf)
    pdf['liter_diesel'], pdf['galon_diesel'] = column_splitter('diesel', pdf)
    
    pdf['company'] = pdf['company'].str.replace('*', '').str.title()
    
    pdf['gas_date'] = pd.to_datetime(pdf['gas_date'])
    pdf['diesel_date'] = pd.to_datetime(pdf['diesel_date'])
    
    return pdf

In [4]:
pdf_files = os.listdir('pdf_files/')

In [6]:
if 'csv_files' not in os.listdir('.'):
    os.makedirs('csv_files')
else:
    os.system('rm -rf csv_files')
    os.makedirs('csv_files')

In [8]:
parsed_files = list()
problematic_files = list()

for file in pdf_files:
    try:
        parsed_pdf = file_parser('pdf_files/' + file)
    
        new_filename = file.replace('.pdf', '.csv')
        parsed_pdf.to_csv('csv_files/' + new_filename)
    
        parsed_files.append(parsed_pdf)
    except:
        print('I\'m causing trouble:' + file)
        problematic_files.append(file)

I'm causing trouble:2013-03-12.pdf
I'm causing trouble:2012-08-24.pdf
I'm causing trouble:2013-03-13.pdf
I'm causing trouble:2012-10-26.pdf
I'm causing trouble:2013-03-27.pdf
I'm causing trouble:2012-10-09.pdf
I'm causing trouble:2012-12-05.pdf
I'm causing trouble:2012-10-11.pdf
I'm causing trouble:2012-09-04.pdf
I'm causing trouble:2012-07-20.pdf
I'm causing trouble:2012-11-14.pdf
I'm causing trouble:2013-02-20.pdf
I'm causing trouble:2012-07-19.pdf
I'm causing trouble:2013-01-15.pdf
I'm causing trouble:2012-12-12.pdf
I'm causing trouble:2013-03-04.pdf
I'm causing trouble:2013-04-02.pdf
I'm causing trouble:2013-02-22.pdf
I'm causing trouble:2012-12-07.pdf
I'm causing trouble:2012-11-29.pdf
I'm causing trouble:2013-05-13.pdf
I'm causing trouble:2013-02-11.pdf
I'm causing trouble:2013-02-21.pdf
I'm causing trouble:2012-09-05.pdf
I'm causing trouble:2013-04-26.pdf
I'm causing trouble:2013-01-09.pdf
I'm causing trouble:2013-04-08.pdf
I'm causing trouble:2013-04-01.pdf
I'm causing trouble:

Error: 


I'm causing trouble:.ipynb_checkpoints
I'm causing trouble:2013-03-07.pdf
I'm causing trouble:2013-01-25.pdf
I'm causing trouble:2012-10-31.pdf
I'm causing trouble:2013-03-19.pdf
I'm causing trouble:2013-02-05.pdf
I'm causing trouble:2013-05-09.pdf
I'm causing trouble:2013-01-28.pdf
I'm causing trouble:2012-07-10.pdf
I'm causing trouble:2012-08-29.pdf
I'm causing trouble:2012-09-25.pdf
I'm causing trouble:2012-08-17.pdf
I'm causing trouble:2012-11-05.pdf
I'm causing trouble:2012-08-30.pdf
I'm causing trouble:2013-02-13.pdf
I'm causing trouble:2013-01-08.pdf
I'm causing trouble:2012-07-09.pdf
I'm causing trouble:2013-03-14.pdf
I'm causing trouble:2012-07-13.pdf
I'm causing trouble:2012-12-11.pdf
I'm causing trouble:2013-03-11.pdf
I'm causing trouble:2012-12-20.pdf
I'm causing trouble:2012-11-30.pdf
I'm causing trouble:2012-12-03.pdf
I'm causing trouble:2013-03-06.pdf
I'm causing trouble:2013-03-25.pdf
I'm causing trouble:2013-04-09.pdf
I'm causing trouble:2012-11-20.pdf
I'm causing trou

In [47]:
parsed_data = pd.concat(parsed_files)

In [49]:
new_date = []
for idx, date in enumerate(parsed_data['gas_date']):
    if date > pd.datetime(2000, 1, 1):
        new_date.append(date)
    else:
        new_date.append(parsed_data['diesel_date'].values[idx])

In [50]:
parsed_data['date'] = new_date

In [51]:
parsed_data.drop('gas_date', axis = 1, inplace = True)
parsed_data.drop('diesel_date', axis = 1, inplace = True)

In [56]:
os.makedirs('data')

In [57]:
parsed_data.to_csv('data/gasoline_prices_PR.csv')